In [ ]:
# ============================
# 📌 Step 1: Import Libraries
# ============================
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

# =====================================
# 📌 Step 2: Load and Preprocess Image
# =====================================
file_path = r"C:\MyFolder\Grind\SVD\dog.jpg"
dog = Image.open(file_path).convert("RGB")

plt.figure(figsize=(6,6))
plt.imshow(dog)
plt.title('Original Image')
plt.axis('off')
plt.show()

file_size = os.path.getsize(file_path)
print(f"Original Image Size: {np.round(file_size/1024)} KB")
print(f"Original Image Matrix Shape: {np.shape(dog)}")

In [ ]:
dog_array = np.array(dog, dtype=np.float64)
dog_gray = np.mean(dog_array, axis=2)   # grayscale
dog_gray = np.array(dog_gray, dtype=np.uint8)

plt.figure(figsize=(6,6))
plt.imshow(dog_gray, cmap='gray')
plt.title('Grayscale Dog')
plt.axis('off')
plt.show()

file_path_gray_dog = r"C:\MyFolder\Grind\SVD\gray_dog.png"
Image.fromarray(dog_gray).save(file_path_gray_dog)

file_size_gray_dog = os.path.getsize(file_path_gray_dog)
print(f"Size of Grayscale Dog: {np.round(file_size_gray_dog/1024)} KB")
print(f"Shape of Grayscale Dog: {dog_gray.shape}")


In [ ]:
# ====================================================
# 📌 Step 3: Apply SVD [Singular Value Decomposition]
# ====================================================
U, s, Vt = np.linalg.svd(dog_gray, full_matrices=False)
print(f'Shape of U = {U.shape}')
print(f'Shape of s = {s.shape}')
print(f'Shape of Vt = {Vt.shape}')


In [ ]:

plt.style.use("dark_background")

# ============================================
# 📌 Step 4: singular values (log scale) Plot
# ============================================
plt.figure(figsize=(10,5))
plt.semilogy(s, color="cyan", lw=2)
plt.title("Singular Values (log scale)", fontsize=15, color="white")
plt.xlabel("Index", fontsize=13, color="white")
plt.ylabel("Value", fontsize=13, color="white")
plt.grid(True, linestyle="--", alpha=0.4)
plt.show()

# ==================================
# 📌 Step 5: Cumulative Energy Plot
# ==================================
energy = np.cumsum(s**2) / np.sum(s**2)

plt.figure(figsize=(10,5))
plt.plot(energy, color="magenta", lw=2, label="Cumulative Energy")
plt.axhline(y=0.9, color='yellow', linestyle='--', alpha=0.7, label="90%")
plt.axhline(y=0.95, color='lime', linestyle='--', alpha=0.7, label="95%")

plt.legend(facecolor="black", edgecolor="white")
plt.title("Cumulative Energy", fontsize=15, color="white")
plt.xlabel("Number of Components (k)", fontsize=13, color="white")
plt.ylabel("Energy Captured", fontsize=13, color="white")
plt.grid(True, linestyle="--", alpha=0.4)
plt.show()


In [ ]:
# ==================================================================
# 📌 Step 6: Rank-1 Approximations / Contribution of Top Components
# ==================================================================
plt.figure(figsize=(12,6))
for i in range(6):
    component = s[i] * np.outer(U[:,i], Vt[i,:])
    plt.subplot(2,3,i+1)
    plt.imshow(component, cmap='gray')
    plt.title(f"Component {i+1}")
    plt.axis('off')
plt.show()


In [ ]:
# ===========================================
# 📌 Step 7: Reconstruction with Different k
# ===========================================
def reconstruct(U, s, Vt, k):
    return (U[:, :k] * s[:k]) @ Vt[:k, :]

ks = [5, 20, 50, 100]

plt.figure(figsize=(12,8))
for i, k in enumerate(ks):
    recon = reconstruct(U, s, Vt, k)
    plt.subplot(2,2,i+1)
    plt.imshow(recon, cmap='gray')
    plt.title(f"Reconstructed with k={k}")
    plt.axis('off')
plt.show()


In [ ]:

# =======================
# 📌 Step 8: Error vs. k
# =======================
errors = []
ks = np.arange(5, 200, 5)

for k in ks:
    recon = (U[:, :k] * s[:k]) @ Vt[:k, :]
    mse = np.mean((dog_gray - recon) ** 2)
    errors.append(mse)

# Apply dark theme
plt.style.use("dark_background")

plt.figure(figsize=(10,6))
plt.plot(ks, errors, marker='o', color='cyan', lw=2, markersize=6, label="Reconstruction Error")

# Highlight important thresholds
plt.axvline(x=20, color='yellow', linestyle='--', alpha=0.7, label="k=20")
plt.axvline(x=50, color='magenta', linestyle='--', alpha=0.7, label="k=50")

# Labels and title
plt.title("Reconstruction Error vs Number of Components", fontsize=16, pad=15, color="white")
plt.xlabel("Number of Components (k)", fontsize=13, color="white")
plt.ylabel("Mean Squared Error (MSE)", fontsize=13, color="white")

plt.legend(facecolor="black", edgecolor="white")
plt.grid(True, linestyle="--", alpha=0.4)

plt.show()


In [ ]:
# ================================
# 📌 Step 9: Visualize Components
# ================================
# --- Compression Info ---
num_pc_components = 50  # you can adjust k
dog_reconstructed = reconstruct(U, s, Vt, num_pc_components)
dog_reconstructed = np.clip(dog_reconstructed, 0, 255).astype(np.uint8)

# Save reconstructed image
reconstructed_dog_path = r"C:\MyFolder\Grind\SVD\dog_reconstructed.png"
Image.fromarray(dog_reconstructed).save(reconstructed_dog_path)

# File sizes
reconstructed_dog_size = os.path.getsize(reconstructed_dog_path)
print(f"Size of Reconstructed Dog (k={num_pc_components}): {np.round(reconstructed_dog_size/1024,1)} KB")
print(f"Size of Greyscale Dog: {np.round(file_size_gray_dog/1024,1)} KB")
print(f"Percent Compression Ratio: {np.round(file_size_gray_dog/reconstructed_dog_size * 100,1)}")

# --- Side by Side Plot in Dark Theme ---
plt.style.use("dark_background")
plt.figure(figsize=(9,4))  # reduced width so no scroll

plt.subplot(1,2,1)
plt.imshow(dog_gray, cmap='gray')
plt.title("Original Grayscale Dog")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(dog_reconstructed, cmap='gray')
plt.title(f"Reconstructed Dog (k={num_pc_components})")
plt.axis('off')

plt.tight_layout()
plt.show()
